In [4]:
import http.client
import urllib.parse
from pprint import pprint
import json

## **Using Glassdoor API**

In [2]:
def glassDoor(company, job_title, location, API_KEY=None):
    conn = http.client.HTTPSConnection("glassdoor-real-time.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
        'x-rapidapi-host': "glassdoor-real-time.p.rapidapi.com"
    }
    # first extract locationId
    params = {
        'query' : location
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/jobs/location?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    res = json.loads(data)
    locationId = res['data'][0]['locationId'] # we got the location ID
    # Now extract salary data
    params = {
        'query' : f'{company} {job_title}',
        'locationId': locationId,
        'limit': 10,
        'page': 1,
        'sort' : 'POPULAR'
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/salaries/search?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    res = json.loads(data)
    return res

In [3]:
result = glassDoor('apple', 'software developer', 'new york')

In [15]:
print(result)

{'data': {'aggregateSalaryResponse': {'numPages': 103, 'queryLocation': {'id': 1132348, 'name': 'New York, NY', 'type': 'CITY'}, 'resultCount': 1024, 'results': [{'basePayStatistics': {'mean': 31.02}, 'currency': {'code': 'USD', 'id': 1}, 'employer': {'counts': {'globalJobCount': {'jobCount': 7410}}, 'id': 1138, 'name': 'Apple Inc.', 'ratings': {'overallRating': 4.1}, 'shortName': 'Apple', 'squareLogoUrl': 'https://media.glassdoor.com/sql/1138/apple-squarelogo-1595530154096.png'}, 'jobTitle': {'gocId': 102425, 'id': 30663, 'text': 'Specialist'}, 'payPeriod': 'HOURLY', 'totalAdditionalPayStatistics': {'mean': 0.0}, 'totalPayStatistics': {'__typename': 'StatisticsResult', 'percentiles': [{'ident': 'P10', 'value': 18.36}, {'ident': 'P25', 'value': 23.53}, {'ident': 'P50', 'value': 31.02}, {'ident': 'P75', 'value': 40.89}, {'ident': 'P90', 'value': 52.41}]}, 'salaryId': 'eyJjaWQiOjExMzIzNDgsInR5cGUiOiJDSVRZIiwiZUlkIjoxMTM4LCJuIjoiQXBwbGUiLCJqdCI6IlNwZWNpYWxpc3QiLCJqaSI6MzA2NjMsImdJIjoxMDI0

In [19]:
      def extract_salary(output_dict):
            results = output_dict['data']['aggregateSalaryResponse']['results']
            company_name = results.get('employer',{}).get('name',{})
            location = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('queryLocation',{}).get('name',{})
            # Initialize results container
            container = []
            # we will traverse through all job entries
            for job in results:
                pay_period = job['payPeriod']
                # we will only process annual salaries
                if pay_period == 'ANNUAL':
                    job_title = job['jobTitle']['text']
                    currency = job['currency']['code']
                    base_mean = job['basePayStatistics']['mean']
                    total_additional_mean = job['totalAdditionalPayStatistics']['mean']
                    percentiles = {p['ident']: p['value'] for p in job['totalPayStatistics']['percentiles']}
                    
                    # salary summary
                    container.append({
                        'company_name': company_name,
                        'job_title': job_title,
                        'location': location,
                        # base salary ranges
                        'min_base_salary': round(base_mean * 0.81, 2),  # estimated ~19% below mean
                        'median_base_salary': round(base_mean, 2),
                        'max_base_salary': round(base_mean * 1.23, 2),  # estimated ~23% above mean
                        # general salary ranges
                        'min_salary': round(percentiles.get('P25', 0), 2),
                        'median_salary': round(percentiles.get('P50', 0), 2),
                        'mean_salary': round(total_additional_mean, 2),
                        'max_salary': round(percentiles.get('P75', 0), 2),
                        'salary_period': 'YEAR',
                        'salary_range': f"{currency} {round(percentiles.get('P25', 0), 2)} - {currency} {round(percentiles.get('P75', 0), 2)}",
                        'salary_currency': currency
                    })
            return container

In [20]:
extract_salary(result)

AttributeError: 'list' object has no attribute 'get'

In [16]:
import json

# Assume this is your parsed JSON object
data = result['data']['aggregateSalaryResponse']['results']

# Initialize results container
all_salaries = {}

# Loop through all job entries
for job in data:
    job_title = job['jobTitle']['text']
    pay_period = job['payPeriod']
    currency = job['currency']['code']
    
    # Only process annual salaries
    if pay_period == 'ANNUAL':
        base_mean = job['basePayStatistics']['mean']
        total_additional_mean = job['totalAdditionalPayStatistics']['mean']
        percentiles = {p['ident']: p['value'] for p in job['totalPayStatistics']['percentiles']}
        
        # Construct the salary summary
        all_salaries[job_title] = {
            'min_base_salary': round(base_mean * 0.81, 2),  # estimated ~19% below mean
            'median_base_salary': round(base_mean, 2),
            'max_base_salary': round(base_mean * 1.23, 2),  # estimated ~23% above mean
            'min_salary': round(percentiles.get('P25', 0), 2),
            'median_salary': round(percentiles.get('P50', 0), 2),
            'mean_salary': round(total_additional_mean, 2),
            'max_salary': round(percentiles.get('P75', 0), 2),
            'salary_period': 'YEAR',
            'salary_range': f"{currency} {round(percentiles.get('P25', 0), 2)} - {currency} {round(percentiles.get('P75', 0), 2)}",
            'salary_currency': currency
        }

# Print nicely
print(json.dumps(all_salaries, indent=2))


{
  "Software Engineer": {
    "min_base_salary": 149867.71,
    "median_base_salary": 185021.86,
    "max_base_salary": 227576.89,
    "min_salary": 228183.85,
    "median_salary": 289326.03,
    "mean_salary": 104304.17,
    "max_salary": 374313.83,
    "salary_period": "YEAR",
    "salary_range": "USD 228183.85 - USD 374313.83",
    "salary_currency": "USD"
  },
  "Sales Associate": {
    "min_base_salary": 60962.76,
    "median_base_salary": 75262.67,
    "max_base_salary": 92573.08,
    "min_salary": 57027.03,
    "median_salary": 75262.67,
    "mean_salary": 0.0,
    "max_salary": 99329.56,
    "salary_period": "YEAR",
    "salary_range": "USD 57027.03 - USD 99329.56",
    "salary_currency": "USD"
  },
  "Manager": {
    "min_base_salary": 102433.6,
    "median_base_salary": 126461.23,
    "max_base_salary": 155547.31,
    "min_salary": 133158.43,
    "median_salary": 176750.59,
    "mean_salary": 50289.36,
    "max_salary": 237968.04,
    "salary_period": "YEAR",
    "salary_ran

'min_base_salary': 149955.72, 
'median_base_salary': 185021.86, 
'max_base_salary': 228287.99, 
'min_salary': 228183.85, 
'median_salary': 289326.03, 
'mean_salary': 104304.17, 
'max_salary': 374313.83, 
'salary_period': 'YEAR', 
'salary_range': 'USD 228183.85 - USD 374313.83', 
'salary_currency': 'USD'}

In [23]:
result.get('data',{}).get('aggregateSalaryResponse',{}).get('results')

{'numPages': 104,
 'queryLocation': {'id': 1132348, 'name': 'New York, NY', 'type': 'CITY'},
 'resultCount': 1040,
 'results': [{'basePayStatistics': {'mean': 160980.7},
   'currency': {'code': 'USD', 'id': 1},
   'employer': {'counts': {'globalJobCount': {'jobCount': 721}},
    'id': 1651,
    'name': 'Microsoft Corporation',
    'ratings': {'overallRating': 4.2},
    'shortName': 'Microsoft',
    'squareLogoUrl': 'https://media.glassdoor.com/sql/1651/microsoft-squarelogo-1479856042252.png'},
   'jobTitle': {'gocId': 100063, 'id': 4279, 'text': 'Software Engineer'},
   'payPeriod': 'ANNUAL',
   'totalAdditionalPayStatistics': {'mean': 57123.0},
   'totalPayStatistics': {'__typename': 'StatisticsResult',
    'percentiles': [{'ident': 'P10', 'value': 151869.73},
     {'ident': 'P25', 'value': 179986.26},
     {'ident': 'P50', 'value': 218103.7},
     {'ident': 'P75', 'value': 268932.58},
     {'ident': 'P90', 'value': 322786.2}]},
   'salaryId': 'eyJjaWQiOjExMzIzNDgsInR5cGUiOiJDSVRZIiwi

In [24]:
# get location
loc = result.get('data',{}).get('aggregateSalaryResponse',{}).get('queryLocation',{}).get('name',{})
loc

'New York, NY'

In [26]:
# explore results
results = result.get('data',{}).get('aggregateSalaryResponse',{}).get('results')
# take 1st result
r1 = results[0]
r1

{'basePayStatistics': {'mean': 160980.7},
 'currency': {'code': 'USD', 'id': 1},
 'employer': {'counts': {'globalJobCount': {'jobCount': 721}},
  'id': 1651,
  'name': 'Microsoft Corporation',
  'ratings': {'overallRating': 4.2},
  'shortName': 'Microsoft',
  'squareLogoUrl': 'https://media.glassdoor.com/sql/1651/microsoft-squarelogo-1479856042252.png'},
 'jobTitle': {'gocId': 100063, 'id': 4279, 'text': 'Software Engineer'},
 'payPeriod': 'ANNUAL',
 'totalAdditionalPayStatistics': {'mean': 57123.0},
 'totalPayStatistics': {'__typename': 'StatisticsResult',
  'percentiles': [{'ident': 'P10', 'value': 151869.73},
   {'ident': 'P25', 'value': 179986.26},
   {'ident': 'P50', 'value': 218103.7},
   {'ident': 'P75', 'value': 268932.58},
   {'ident': 'P90', 'value': 322786.2}]},
 'salaryId': 'eyJjaWQiOjExMzIzNDgsInR5cGUiOiJDSVRZIiwiZUlkIjoxNjUxLCJuIjoiTWljcm9zb2Z0IiwianQiOiJTb2Z0d2FyZSBFbmdpbmVlciIsImppIjo0Mjc5LCJnSSI6MTAwMDYzfQ=='}

In [81]:
# get the base mean
r1.get('basePayStatistics',{}).get('mean',{})
# get the currency
r1.get('currency', {}).get('code',{})
# get the employer
r1.get('employer', {}).get('name',{})
# get the ratings
r1.get('employer', {}).get('ratings',{}).get('overallRating',{})
# get the job title
r1.get('jobTitle').get('text', {})
# get the pay period
r1.get('payPeriod', {})
# get the total additional mean
r1.get('totalAdditionalPayStatistics',{}).get('mean', {})
# get the percentiles
r1.get('totalPayStatistics',{}).get('percentiles',{})
percentiles = r1.get('totalPayStatistics',{}).get('percentiles',{})

p25 = next((p['value'] for p in percentiles if p.get('ident') == 'p_25'), None)
p50 = next((p['value'] for p in percentiles if p.get('ident') == 'p_50'), None)
p75 = next((p['value'] for p in percentiles if p.get('ident') == 'p_75'), None)
print(percentiles)
print([p25, p50, p75])

[{'ident': 'P10', 'value': 151869.73}, {'ident': 'P25', 'value': 179986.26}, {'ident': 'P50', 'value': 218103.7}, {'ident': 'P75', 'value': 268932.58}, {'ident': 'P90', 'value': 322786.2}]
[None, None, None]


## Written by Me

In [85]:
def extract_salary(output_dict):
    # container to store output
    output = []
    # get the location
    location = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('queryLocation',{}).get('name',{})
    # go to the results section
    results = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('results')
    # traverse each job
    for result in results:
        # get the employer
        employer = result.get('employer', {}).get('name',{}) 
        # get the job title
        job_title = result.get('jobTitle').get('text', {})
        # get the salary currency
        currency = result.get('currency', {}).get('code',{})
        # get the salary period
        period = result.get('payPeriod', {})
        # get the base mean
        base_mean = result.get('basePayStatistics',{}).get('mean',{})
        # get the percentiles (only useful to us - p25, p50, p75)
        percentiles = result.get('totalPayStatistics',{}).get('percentiles',{}) # we get a list here
        p25 = next((p['value'] for p in percentiles if p.get('ident') == 'P25'), None)
        p50 = next((p['value'] for p in percentiles if p.get('ident') == 'P50'), None)
        p75 = next((p['value'] for p in percentiles if p.get('ident') == 'P75'), None)
        # get the salary range (p25 - p75) since, p50 is median
        # get the median salary
        median_salary = p50
        # get mean salary
        mean_salary = result.get('totalAdditionalPayStatistics',{}).get('mean', {})
        # get the ratings
        rating = result.get('employer', {}).get('ratings',{}).get('overallRating',{})
        entry = {
            'employer': employer,
            'job_title': job_title,
            'location': location,
            'currency': currency,
            'period': period,
            'salary_range': f'{currency} {p25} - {currency} {p75}',
            'base_salary': f'{currency} {base_mean}',
            'median_salary': f'{currency} {median_salary}',
            'mean_salary': f'{currency} {mean_salary}',
            'company_rating': rating
        }
        output.append(entry)
    return output
'''
{'company': 'microsoft',
  'job_title': 'Software Engineer',
  'location': 'new york',
  'salary_currency': 'USD',
  'salary_period': 'annual',
  'salary_range': 'N/A',
  'median_salary': 'N/A',
  'mean_salary': 'N/A',
  'company_rating': 'N/A'}
'''

"\n{'company': 'microsoft',\n  'job_title': 'Software Engineer',\n  'location': 'new york',\n  'salary_currency': 'USD',\n  'salary_period': 'annual',\n  'salary_range': 'N/A',\n  'median_salary': 'N/A',\n  'mean_salary': 'N/A',\n  'company_rating': 'N/A'}\n"

In [86]:
salary_data = extract_salary(result)

### **[OUTPUT]**

In [87]:
salary_data

[{'employer': 'Microsoft Corporation',
  'job_title': 'Software Engineer',
  'location': 'New York, NY',
  'currency': 'USD',
  'period': 'ANNUAL',
  'salary_range': 'USD 179986.26 - USD 268932.58',
  'base_salary': 'USD 160980.7',
  'median_salary': 'USD 218103.7',
  'mean_salary': 'USD 57123.0',
  'company_rating': 4.2},
 {'employer': 'Microsoft Corporation',
  'job_title': 'Product Manager',
  'location': 'New York, NY',
  'currency': 'USD',
  'period': 'ANNUAL',
  'salary_range': 'USD 196016.29 - USD 286820.51',
  'base_salary': 'USD 171008.83',
  'median_salary': 'USD 234861.99',
  'mean_salary': 'USD 63853.16',
  'company_rating': 4.2},
 {'employer': 'Microsoft Corporation',
  'job_title': 'Senior Software Engineer',
  'location': 'New York, NY',
  'currency': 'USD',
  'period': 'ANNUAL',
  'salary_range': 'USD 236292.54 - USD 343486.76',
  'base_salary': 'USD 198162.34',
  'median_salary': 'USD 281862.05',
  'mean_salary': 'USD 83699.71',
  'company_rating': 4.2},
 {'employer': 

## **using Jsearch API**

In [128]:
def jsearch(company, job_title, location, API_KEY=None):
    conn = http.client.HTTPSConnection("jsearch.p.rapidapi.com")
    url = "jsearch.p.rapidapi.com"
    headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "jsearch.p.rapidapi.com"
    }
    params = {
    'company': company,
    'job_title': job_title,
    'location' : location,
    'page': 1,
    'location_type': 'ANY',
    'years_of_experience': 'ALL',
    'num_pages' : 10
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/company-job-salary?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read() # -> type() : str
    # get the result in dictionary form 
    res = json.loads(data)
    salary_data = res['data'][0]
    return {
        "employer": salary_data["company"],
        "job_title": salary_data["job_title"],
        "location": salary_data["location"],
        "currency": salary_data["salary_currency"],
        'period': salary_data['salary_period'],
        "salary_range": f"{salary_data['salary_currency']} {int(salary_data['min_salary']):,} - {int(salary_data['max_salary']):,} per {salary_data['salary_period'].lower()}",
        "base_salary": f"{salary_data['salary_currency']} {salary_data['median_base_salary']}",
        "median_salary": f"{salary_data['salary_currency']} {int(salary_data['median_salary']):,}",
        "company_rating": None
    }


In [129]:
salary_data = jsearch('apple', 'software engineer', 'new york')

### **[OUTPUT]**

In [130]:
salary_data

{'employer': 'Apple',
 'job_title': 'Software Engineer',
 'location': 'New York City, NY',
 'currency': 'USD',
 'period': 'YEAR',
 'salary_range': 'USD 228,183 - 374,313 per year',
 'base_salary': 'USD 185021.86',
 'median_salary': 'USD 289,326',
 'company_rating': None}

## **Job-data-API**

In [8]:
from pprint import pprint
sample = {'status': 'OK', 'request_id': '92815ef3-ac3d-4a08-b64a-1c76b3a334f0', 'data': [{'location': 'New York City, NY', 'job_title': 'Software Developer', 'company': 'Amazon', 'min_salary': 177650.22, 'max_salary': 276253.84, 'median_salary': 219151.35, 'min_base_salary': 120605.66, 'max_base_salary': 169770.67, 'median_base_salary': 143091.94, 'min_additional_pay': 57044.56, 'max_additional_pay': 106483.17, 'median_additional_pay': 76059.41, 'salary_period': 'YEAR', 'salary_currency': 'USD', 'salary_count': 92, 'confidence': 'CONFIDENT', 'result': {'__typename': 'AggregatedSalaryEstimatesResult', 'basePayStatistics': {'__typename': 'StatisticsResult', 'max': 169770.67, 'mean': 143091.94, 'min': 120605.66, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 103418.91}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 120605.66}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 143091.94}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 169770.67}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 197984.15}]}, 'confidence': 'CONFIDENT', 'currency': {'__typename': 'Currency', 'code': 'USD'}, 'employer': {'__typename': 'Employer', 'counts': {'__typename': 'EmployerCounts', 'globalJobCount': {'__typename': 'EmployerJobCountHolder', 'jobCount': 28589}}, 'id': 6036, 'links': {'__typename': 'EiEmployerLinks', 'jobsUrl': '/Jobs/Amazon-Jobs-E6036.htm'}, 'ratings': {'__typename': 'DefaultEmployerRatings', 'overallRating': 3.6}, 'shortName': 'Amazon', 'squareLogoUrl': 'https://media.glassdoor.com/sql/6036/amazon-squarelogo-1552847650117.png'}, 'jobTitle': {'__typename': 'JobTitle', 'id': 30270, 'text': 'Software Developer'}, 'obscuring': 'NONE', 'payPeriod': 'ANNUAL', 'salariesEmploymentStatus': 'REGULAR', 'salaryCount': 92, 'totalAdditionalPayStatistics': {'__typename': 'StatisticsResult', 'max': 106483.17, 'mean': 76059.41, 'min': 57044.56, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 44114.46}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 57044.56}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 76059.41}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 106483.17}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 139188.72}]}, 'totalPayInsights': None, 'totalPayStatistics': {'__typename': 'StatisticsResult', 'max': 276253.84, 'mean': 219151.35, 'min': 177650.22, 'percentiles': [{'__typename': 'Percentile', 'ident': 'P10', 'value': 147533.37}, {'__typename': 'Percentile', 'ident': 'P25', 'value': 177650.22}, {'__typename': 'Percentile', 'ident': 'P50', 'value': 219151.35}, {'__typename': 'Percentile', 'ident': 'P75', 'value': 276253.84}, {'__typename': 'Percentile', 'ident': 'P90', 'value': 337172.87}]}}}]}
pprint(sample)

{'data': [{'company': 'Amazon',
           'confidence': 'CONFIDENT',
           'job_title': 'Software Developer',
           'location': 'New York City, NY',
           'max_additional_pay': 106483.17,
           'max_base_salary': 169770.67,
           'max_salary': 276253.84,
           'median_additional_pay': 76059.41,
           'median_base_salary': 143091.94,
           'median_salary': 219151.35,
           'min_additional_pay': 57044.56,
           'min_base_salary': 120605.66,
           'min_salary': 177650.22,
           'result': {'__typename': 'AggregatedSalaryEstimatesResult',
                      'basePayStatistics': {'__typename': 'StatisticsResult',
                                            'max': 169770.67,
                                            'mean': 143091.94,
                                            'min': 120605.66,
                                            'percentiles': [{'__typename': 'Percentile',
                                             

In [1]:
result = {'data': [{'company': 'Amazon',
           'confidence': 'CONFIDENT',
           'job_title': 'Software Developer',
           'location': 'New York City, NY',
           'max_additional_pay': 106483.17,
           'max_base_salary': 169770.67,
           'max_salary': 276253.84,
           'median_additional_pay': 76059.41,
           'median_base_salary': 143091.94,
           'median_salary': 219151.35,
           'min_additional_pay': 57044.56,
           'min_base_salary': 120605.66,
           'min_salary': 177650.22,
           'result': {'__typename': 'AggregatedSalaryEstimatesResult',
                      'basePayStatistics': {'__typename': 'StatisticsResult',
                                            'max': 169770.67,
                                            'mean': 143091.94,
                                            'min': 120605.66,
                                            'percentiles': [{'__typename': 'Percentile',
                                                             'ident': 'P10',
                                                             'value': 103418.91},
                                                            {'__typename': 'Percentile',
                                                             'ident': 'P25',
                                                             'value': 120605.66},
                                                            {'__typename': 'Percentile',
                                                             'ident': 'P50',
                                                             'value': 143091.94},
                                                            {'__typename': 'Percentile',
                                                             'ident': 'P75',
                                                             'value': 169770.67},
                                                            {'__typename': 'Percentile',
                                                             'ident': 'P90',
                                                             'value': 197984.15}]},
                      'confidence': 'CONFIDENT',
                      'currency': {'__typename': 'Currency', 'code': 'USD'},
                      'employer': {'__typename': 'Employer',
                                   'counts': {'__typename': 'EmployerCounts',
                                              'globalJobCount': {'__typename': 'EmployerJobCountHolder',
                                                                 'jobCount': 28589}},
                                   'id': 6036,
                                   'links': {'__typename': 'EiEmployerLinks',
                                             'jobsUrl': '/Jobs/Amazon-Jobs-E6036.htm'},
                                   'ratings': {'__typename': 'DefaultEmployerRatings',
                                               'overallRating': 3.6},
                                   'shortName': 'Amazon',
                                   'squareLogoUrl': 'https://media.glassdoor.com/sql/6036/amazon-squarelogo-1552847650117.png'},
                      'jobTitle': {'__typename': 'JobTitle',
                                   'id': 30270,
                                   'text': 'Software Developer'},
                      'obscuring': 'NONE',
                      'payPeriod': 'ANNUAL',
                      'salariesEmploymentStatus': 'REGULAR',
                      'salaryCount': 92,
                      'totalAdditionalPayStatistics': {'__typename': 'StatisticsResult',
                                                       'max': 106483.17,
                                                       'mean': 76059.41,
                                                       'min': 57044.56,
                                                       'percentiles': [{'__typename': 'Percentile',
                                                                        'ident': 'P10',
                                                                        'value': 44114.46},
                                                                       {'__typename': 'Percentile',
                                                                        'ident': 'P25',
                                                                        'value': 57044.56},
                                                                       {'__typename': 'Percentile',
                                                                        'ident': 'P50',
                                                                        'value': 76059.41},
                                                                       {'__typename': 'Percentile',
                                                                        'ident': 'P75',
                                                                        'value': 106483.17},
                                                                       {'__typename': 'Percentile',
                                                                        'ident': 'P90',
                                                                        'value': 139188.72}]},
                      'totalPayInsights': None,
                      'totalPayStatistics': {'__typename': 'StatisticsResult',
                                             'max': 276253.84,
                                             'mean': 219151.35,
                                             'min': 177650.22,
                                             'percentiles': [{'__typename': 'Percentile',
                                                              'ident': 'P10',
                                                              'value': 147533.37},
                                                             {'__typename': 'Percentile',
                                                              'ident': 'P25',
                                                              'value': 177650.22},
                                                             {'__typename': 'Percentile',
                                                              'ident': 'P50',
                                                              'value': 219151.35},
                                                             {'__typename': 'Percentile',
                                                              'ident': 'P75',
                                                              'value': 276253.84},
                                                             {'__typename': 'Percentile',
                                                              'ident': 'P90',
                                                              'value': 337172.87}]}},
           'salary_count': 92,
           'salary_currency': 'USD',
           'salary_period': 'YEAR'}],
 'request_id': '92815ef3-ac3d-4a08-b64a-1c76b3a334f0',
 'status': 'OK'}

In [2]:
result

{'data': [{'company': 'Amazon',
   'confidence': 'CONFIDENT',
   'job_title': 'Software Developer',
   'location': 'New York City, NY',
   'max_additional_pay': 106483.17,
   'max_base_salary': 169770.67,
   'max_salary': 276253.84,
   'median_additional_pay': 76059.41,
   'median_base_salary': 143091.94,
   'median_salary': 219151.35,
   'min_additional_pay': 57044.56,
   'min_base_salary': 120605.66,
   'min_salary': 177650.22,
   'result': {'__typename': 'AggregatedSalaryEstimatesResult',
    'basePayStatistics': {'__typename': 'StatisticsResult',
     'max': 169770.67,
     'mean': 143091.94,
     'min': 120605.66,
     'percentiles': [{'__typename': 'Percentile',
       'ident': 'P10',
       'value': 103418.91},
      {'__typename': 'Percentile', 'ident': 'P25', 'value': 120605.66},
      {'__typename': 'Percentile', 'ident': 'P50', 'value': 143091.94},
      {'__typename': 'Percentile', 'ident': 'P75', 'value': 169770.67},
      {'__typename': 'Percentile', 'ident': 'P90', 'valu

In [3]:
r1 = result['data'][0]
r1

{'company': 'Amazon',
 'confidence': 'CONFIDENT',
 'job_title': 'Software Developer',
 'location': 'New York City, NY',
 'max_additional_pay': 106483.17,
 'max_base_salary': 169770.67,
 'max_salary': 276253.84,
 'median_additional_pay': 76059.41,
 'median_base_salary': 143091.94,
 'median_salary': 219151.35,
 'min_additional_pay': 57044.56,
 'min_base_salary': 120605.66,
 'min_salary': 177650.22,
 'result': {'__typename': 'AggregatedSalaryEstimatesResult',
  'basePayStatistics': {'__typename': 'StatisticsResult',
   'max': 169770.67,
   'mean': 143091.94,
   'min': 120605.66,
   'percentiles': [{'__typename': 'Percentile',
     'ident': 'P10',
     'value': 103418.91},
    {'__typename': 'Percentile', 'ident': 'P25', 'value': 120605.66},
    {'__typename': 'Percentile', 'ident': 'P50', 'value': 143091.94},
    {'__typename': 'Percentile', 'ident': 'P75', 'value': 169770.67},
    {'__typename': 'Percentile', 'ident': 'P90', 'value': 197984.15}]},
  'confidence': 'CONFIDENT',
  'currency

In [6]:
# get company name
r1.get('company')
# get job title
r1.get('job_title')
# get the location
r1.get('location')
# get 

'New York City, NY'

In [11]:
def extract_salary(output_dict):
    # container to store output
    output = []
    # get the location
    # location = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('queryLocation',{}).get('name',{})
    # go to the results section
    results = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('results')
    # traverse each job
    for result in results:
        # get the employer
        employer = result.get('employer', {}).get('name',{}) 
        # get the job title
        job_title = result.get('jobTitle').get('text', {})
        # get the salary currency
        currency = result.get('currency', {}).get('code',{})
        # get the salary period
        period = result.get('payPeriod', {})
        # get the base mean
        base_mean = result.get('basePayStatistics',{}).get('mean',{})
        # get the percentiles (only useful to us - p25, p50, p75)
        percentiles = result.get('totalPayStatistics',{}).get('percentiles',{}) # we get a list here
        p25 = next((p['value'] for p in percentiles if p.get('ident') == 'P25'), None)
        p50 = next((p['value'] for p in percentiles if p.get('ident') == 'P50'), None)
        p75 = next((p['value'] for p in percentiles if p.get('ident') == 'P75'), None)
        # get the salary range (p25 - p75) since, p50 is median
        # get the median salary
        median_salary = p50
        # get mean salary
        mean_salary = result.get('totalAdditionalPayStatistics',{}).get('mean', {})
        # get the ratings
        rating = result.get('employer', {}).get('ratings',{}).get('overallRating',{})
        entry = {
            'employer': employer,
            'job_title': job_title,
            'location': location,
            'currency': currency,
            'period': period,
            'salary_range': f'{currency} {p25} - {currency} {p75}',
            'base_salary': f'{currency} {base_mean}',
            'median_salary': f'{currency} {median_salary}',
            'mean_salary': f'{currency} {mean_salary}',
            'company_rating': rating
        }
        output.append(entry)
    return output

In [10]:
op = extract_salary(sample)
op

AttributeError: 'list' object has no attribute 'get'

In [ ]:
def job_salary_data_api(company, job_title, location, API_KEY=None):
    conn = http.client.HTTPSConnection("job-salary-data.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
        'x-rapidapi-host': "job-salary-data.p.rapidapi.com"
    }
    params = {
        'company': company,
        'job_title': job_title,
        'location': location,
        'location_type': 'ANY',
        'years_of_experience': 'ALL'
    }
    url_safe_query = urllib.parse.urlencode(params)
    conn.request("GET", f"/company-job-salary?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    res = json.loads(data)
    salary_data = res['data'][0]
    # return {
    #     'employer': salary_data['company'],
    #     'job_title': salary_data['job_title'],
    #     'location': salary_data['location'],
    #     'currency': salary_data['salary_currency'],
    #     'period': salary_data['salary_period'],
    #     'salary_range' : f'{salary_data['salary_currency']} {salary_data['min_salary']} - {salary_data['salary_currency']} {salary_data['max_salary']}',
    #     'median_salary': f'{salary_data['salary_currency']} {salary_data['median_salary']}',
    #     'mean_salary': f'{salary_data['salary_currency']} {round(salary_data['result']['totalPayStatistics']['mean'])}',
    #     'company_rating': salary_data['result']['employer']['ratings']['overallRating']
    #     # 'confidence': salary_data['confidence'],
    #     # 'min_salary': salary_data['min_salary'],
    #     # 'max_salary': salary_data['max_salary'],
    #     # 'salary_count': salary_data['salary_count'],
    # }

## For CareerJet API

In [8]:
import requests

url = "http://public.api.careerjet.net/search"

params = {
    "locale_code": "en_US",
    "keywords": "software engineer Amazon",
    "location": "new york",
    "affid": "09a7e99d93482b93310a29944476cd33",  
    "user_ip": "1.2.3.4",        
    "user_agent": "Mozilla/5.0", 
    "pagesize": 10,              
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

In [31]:
def extract_generalized_jobs(data):
    salaries = []
    job_list = []
    salary_period = {
        'Y': 'YEAR',
        'A': 'ANNUM',
        'M': 'MONTH',
        'W': 'WEEK',
        'D': 'DAY',
        'H': 'HOUR'
    }
    
    for job in data.get("jobs", []):
        title = job.get("title", "")
        job_list.append(title)
        try:
            min_salary = float(job.get("salary_min", 0))
            max_salary = float(job.get("salary_max", 0))

            if min_salary:
                salaries.append(min_salary)
            if max_salary and max_salary != min_salary:
                salaries.append(max_salary)
        except (ValueError, TypeError):
            continue
    
    salaries.sort()
    currency = job.get("salary_currency_code")

    salary_range = f'{currency} {min(salaries):.2f} - {currency} {max(salaries):.2f}'
    base_salary = f'{currency} {min(salaries):.2f}'
    mean_salary = sum(salaries) / len(salaries)
    median_salary = (
        salaries[len(salaries) // 2]
        if len(salaries) % 2 == 1
        else (salaries[len(salaries) // 2 - 1] + salaries[len(salaries) // 2]) / 2
    )
    job_info = {
        "employer": job.get("company"),
        # "job_title": job.get("title", "").split(",")[0],  # I removed extra qualifiers like amazon-ADS
        "job_title": job_list,
        "location": job.get("locations"),
        "currency": job.get("salary_currency_code"),
        "period": salary_period.get(job.get("salary_type"), job.get("salary_type")),
        "salary_range": salary_range,
        "base_salary": base_salary,
        "median_salary": f'{currency} {median_salary:.2f}',
        "mean_salary": f'{currency} {mean_salary:.2f}',
        "company_rating": None
        # "posted_date": job.get("date"),
        # "job_url": job.get("url")
    }

    return job_info

In [ ]:
def compute_collective_salary_summary(job_data):
    salaries = []
    for job in job_data.get("jobs", []):
        try:
            min_salary = float(job.get("salary_min", 0))
            max_salary = float(job.get("salary_max", 0))

            if min_salary:
                salaries.append(min_salary)
            if max_salary and max_salary != min_salary:
                salaries.append(max_salary)
        except (ValueError, TypeError):
            continue

    if not salaries:
        return {
            'salary_range': 'N/A',
            'base_salary': 'N/A',
            'median_salary': 'N/A',
            'mean_salary': 'N/A',
        }

    salaries.sort()
    currency = job_data['jobs'][0].get('salary_currency_code', 'USD') if job_data['jobs'] else 'USD'

    salary_range = f'{currency} {min(salaries):.2f} - {currency} {max(salaries):.2f}'
    base_salary = f'{currency} {min(salaries):.2f}'
    mean_salary = sum(salaries) / len(salaries)
    median_salary = (
        salaries[len(salaries) // 2]
        if len(salaries) % 2 == 1
        else (salaries[len(salaries) // 2 - 1] + salaries[len(salaries) // 2]) / 2
    )

    return {
        'salary_range': salary_range,
        'base_salary': base_salary,
        'median_salary': f'{currency} {median_salary:.2f}',
        'mean_salary': f'{currency} {mean_salary:.2f}',
    }

## **Output Format :-**
```
{'employer': 'Microsoft Corporation',
  'job_title': 'Software Engineer',
  'location': 'New York, NY',
  'currency': 'USD',
  'period': 'ANNUAL',
  'salary_range': 'USD 179986.26 - USD 268932.58',
  'base_salary': 'USD 160980.7',
  'median_salary': 'USD 218103.7',
  'mean_salary': 'USD 57123.0',
  'company_rating': 4.2}
```

In [32]:
cleaned_jobs = extract_generalized_jobs(data)

cleaned_jobs

{'employer': 'Amazon',
 'job_title': ['Senior Software Engineer (SDEIII), Amazon Ads',
  'Senior Software Engineer, Amazon Autos',
  'Software Development Engineer - Amazon Ads, Recommendation Experience, Amazon Ads',
  'Software Development Engineer II, Amazon Privacy Services',
  'Software Development Engineer, Partner Experience and Growth, Amazon Ads',
  'Software Development Engineer, Amazon Connect',
  'Software Dev Engineer II, Live Events Amazon Advertising',
  'Software Development Engineer II, Amazon Privacy Services',
  'Software Dev Engineer, Amazon',
  'Software Development Engineer/Full Stack developer, Amazon Ads, Campaign Creation, Campaign Creation (Amazon Ads)'],
 'location': 'New York City, NY',
 'currency': 'USD',
 'period': 'YEAR',
 'salary_range': 'USD 129300.00 - USD 151300.00',
 'base_salary': 'USD 129300.00',
 'median_salary': 'USD 129300.00',
 'mean_salary': 'USD 133700.00',
 'company_rating': None}

In [50]:
def extract_job_summary(output_dict):
    salaries = []
    job_list = []
    salary_period = {
        'Y': 'YEAR',
        'A': 'ANNUM',
        'M': 'MONTH',
        'W': 'WEEK',
        'D': 'DAY',
        'H': 'HOUR'
    }
    target = output_dict.get("jobs", [])
    for job in target:
        title = job.get("title", "")
        job_list.append(title)
        try:
            min_salary = float(job.get("salary_min", 0))
            max_salary = float(job.get("salary_max", 0))

            if min_salary:
                salaries.append(min_salary)
            if max_salary and max_salary != min_salary:
                salaries.append(max_salary)
        except (ValueError, TypeError):
            continue
    
    salaries.sort()
    currency = output_dict.get("salary_currency_code")
    salary_range = f'{currency} {min(salaries):.2f} - {currency} {max(salaries):.2f}'
    base_salary = f'{currency} {min(salaries):.2f}'
    mean_salary = sum(salaries) / len(salaries)
    median_salary = (
        salaries[len(salaries) // 2]
        if len(salaries) % 2 == 1
        else (salaries[len(salaries) // 2 - 1] + salaries[len(salaries) // 2]) / 2
    )
    job_info = {
        # p25 = next((p['value'] for p in percentiles if p.get('ident') == 'P25'), None)
        "employer": next((p['company'] for p in output_dict.get('jobs', [])), None),  #output_dict.get("jobs", []).get("company"),
        # "job_title": job.get("title", "").split(",")[0],  # I removed extra qualifiers like amazon-ADS
        "job_title": job_list,
        "location": next(p['locations'] for p in output_dict.get("jobs", [])),   #  output_dict.get("jobs", []).get("locations"),
        "currency": next(p['salary_currency_code'] for p in output_dict.get("jobs", [])),  # output_dict.get("jobs", []).get("salary_currency_code"),
        "period": salary_period.get(job.get("salary_type"), job.get("salary_type")),
        "salary_range": salary_range,
        "base_salary": base_salary,
        "median_salary": f'{currency} {median_salary:.2f}',
        "mean_salary": f'{currency} {mean_salary:.2f}',
        "company_rating": None
    }
    return job_info

In [51]:
sample = {'jobs': [{'salary_min': '151300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Wed, 21 May 2025 06:31:35 GMT',
   'description': '. This is an opportunity to make a significant impact on the future of the <b>Amazon</b> vision.   As a Senior <b>Software</b> <b>Engineer</b>, you will drive the...DESCRIPTION   <b>Amazon</b> is investing heavily in building a world class advertising business and we are responsible...',
   'salary_currency_code': 'USD',
   'salary': '$151300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-194d417e480a06015554730c0e1519061745542d0714434558485f663c01410057021a044e220b471d060c1658296e48430d001c07314e04010f0b021722270d071a45591d7e42021b1946064561290c0f1d0a4e542d0714434558485f655e58104c13/d444ebe9994582edd3a15b3fe94820b2.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Senior Software Engineer (SDEIII), Amazon Ads',
   'salary_max': '151300',
   'company': 'Amazon'},
  {'salary_min': '151300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Thu, 15 May 2025 04:18:27 GMT',
   'description': ' <b>engineer</b> to design and implement the technology for a new <b>Amazon</b> business. We are looking for a Sr. <b>Software</b> Development... coverage and full CI/CD pipelines.   A day in the life  As a Sr. <b>Software</b> Development <b>Engineer</b> on this team, you will create...',
   'salary_currency_code': 'USD',
   'salary': '$151300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-194d417e480a06015554730c0e1519061745542d0714434558485f663c01410057021a044e220b471d060c1658296e48430d001c07314e04010f0b021722270d071a45591d7e42021b1946064561290c0f1d0a4e542d0714434558485f655e58104c13/8bd7578fdf5250dd682dccb3fdecd81a.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Senior Software Engineer, Amazon Autos',
   'salary_max': '151300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Thu, 01 May 2025 22:02:26 GMT',
   'description': 'DESCRIPTION   <b>Amazon</b> Advertising operates at the intersection of eCommerce and advertising, offering a rich array.... We help advertisers reach <b>Amazon</b> customers on <b>Amazon</b>.com, across our other owned and operated sites, on other high quality...',
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1c4e417e42021b19460645432c041802094f04050c1d5e0b78434a0d010b4206223007071a10045211482c1d4d42534848166d3d4812541409130b472145020d051c5a46584359662b0b51114c0c180c0b09110031060e1a444e585f2f2101094e1a45061a412b1f154506010c1d494e3c1c1b53575d/fb7d30a595104c08b08906312346c3d8.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer - Amazon Ads, Recommendation Experience, Amazon Ads',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Thu, 22 May 2025 04:55:54 GMT',
   'description': " looking for talented engineers to join <b>Amazon</b> Privacy Services, where we build and operate critical privacy infrastructure at massive scale....   Our team designs and implements privacy-preserving services that are fundamental to <b>Amazon</b>'s operations. We work on systems...",
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1c13417e42021b19460645432c041802094f04050c1d5e0b78434a0d010b4206223007071a10045211482c1d4d42534848164f276e76730c0e1519061745542d0714434558485f663c01410057021a044e23005611040603474e53592f200a1842184f130504001345651a0f001d4f4e4f2c1c52585614/26f089b1d55dc9cfe62a7426e19e87ef.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer II, Amazon Privacy Services',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Wed, 21 May 2025 02:10:43 GMT',
   'description': ' problems with extensible <b>software</b> solutions . If you are interested in being part of <b>Amazon</b> Advertising and building services...DESCRIPTION   <b>Amazon</b> Advertising operates at the intersection of e-Commerce and Advertising, offering a range...',
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1c4e417e42021b19460645432c041802094f04050c1d5e0b78434a0d010b4206223007071a10045211482c1d4d42534848166d3d4812541409130b472145020d051c5a46584359662b0b51114c0c180c0b09110031060e1a444e585f2f2101094e1a45061a412b1f154506010c1d494e3c1c1b53575d/41138e39af55959aa58c0cbde7fc6c3f.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer, Partner Experience and Growth, Amazon Ads',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Sat, 17 May 2025 23:25:44 GMT',
   'description': " <b>Amazon</b>'s unique experience and expertise, that are used by millions of companies worldwide to manage day-to-day operations... that solve enduring business challenges. We blend vision with curiosity and <b>Amazon</b>'s real-world experience to build opinionated...",
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-4b1f417e42021b19460645432c041802094f04050c1d5e0b78434a0d010b4206223007071a10045211482c1d4d42534848166d3d4812541409130b472145020d051c5a46584359662b0b51114c0c180c0b09110031060e1a444e585f2f25020f5d1b4e432d0f090e0b45111a68421c1c051e/ea22fcd7a9f042d2a3af16d2aa1a3f3d.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer, Amazon Connect',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Fri, 16 May 2025 07:11:50 GMT',
   'description': '.   As part of <b>Amazon</b> Advertising, we build ad products that help advertisers reach their target audiences on worldwide live... events properties. Our <b>software</b> development engineers architect and design sophisticated distributed <b>software</b> systems...',
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1819417e42021b19460645432d0f090e0b45111a493a63296e424b10180f55110026060607090045066a251a5c4e1d685b01011a5476650d0f0800020052542120716b465c57420a4f2b4913490d0d041c66541643505a/e78aa5b7bbdc1c4296efa61258152615.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Dev Engineer II, Live Events Amazon Advertising',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Fri, 16 May 2025 02:41:43 GMT',
   'description': " looking for talented engineers to join <b>Amazon</b> Privacy Services, where we build and operate critical privacy infrastructure at massive scale....   Our team designs and implements privacy-preserving services that are fundamental to <b>Amazon</b>'s operations. We work on systems...",
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1c13417e42021b19460645432c041802094f04050c1d5e0b78434a0d010b4206223007071a10045211482c1d4d42534848164f276e76730c0e1519061745542d0714434558485f663c01410057021a044e23005611040603474e53592f200a1842184f130504001345651a0f001d4f4e4f2c1c52585614/e6ab41eee9c774e3d0aabe51d6cbc7f6.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer II, Amazon Privacy Services',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Thu, 08 May 2025 23:49:31 GMT',
   'description': " needle?   At <b>Amazon</b>, we hire the best minds in technology to innovate and build on behalf of our customers. The focus... we have on our customers is why we are one of the world's most beloved brands - customer obsession is part of our company DNA. Our <b>Software</b>...",
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-4e1e417e42021b19460645432d0f090e0b45111a6b32474a474243442a00401d4e060d136c340a46001f08014f0b79485b662a00401d4e060d136c340a46001f08014f2a0c1b1a5c5c/6019f6ba7baa3e95f04eb064e6bd03e3.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Dev Engineer, Amazon',
   'salary_max': '129300',
   'company': 'Amazon'},
  {'salary_min': '129300',
   'locations': 'New York City, NY',
   'salary_type': 'Y',
   'date': 'Thu, 24 Apr 2025 23:02:08 GMT',
   'description': "DESCRIPTION   The Forge team provides tools, services and frameworks for one of <b>Amazon</b>'s fastest growing businesses...), as well as partners, agencies, and <b>Amazon</b>'s marketing and sales teams, who operate globally and across all device types...",
   'salary_currency_code': 'USD',
   'salary': '$129300 per year',
   'site': '',
   'url': 'https://jobviewtrack.com/en-us/job-1c1f416b5808034e7400410003412a0213451807191658296e424b10180f551100270d170b0b0a50190d07070a6e534a440a0a0b55766616040d4e34114117034936444c544348011d6c741b46171f001c024564111e0c1f455b585f0d2101094e1a45061a632812094c543b1d1249401d6948120a0248044d0606156f5653174c5b/da6a9b7031773d401b5a567129f90bf5.html?affid=09a7e99d93482b93310a29944476cd33',
   'title': 'Software Development Engineer/Full Stack developer, Amazon Ads, Campaign Creation, Campaign Creation (Amazon Ads)',
   'salary_max': '129300',
   'company': 'Amazon'}],
 'hits': 123,
 'response_time': 0.204469919204712,
 'type': 'JOBS',
 'pages': 13}

In [52]:
ans = extract_job_summary(sample)
ans

{'employer': 'Amazon',
 'job_title': ['Senior Software Engineer (SDEIII), Amazon Ads',
  'Senior Software Engineer, Amazon Autos',
  'Software Development Engineer - Amazon Ads, Recommendation Experience, Amazon Ads',
  'Software Development Engineer II, Amazon Privacy Services',
  'Software Development Engineer, Partner Experience and Growth, Amazon Ads',
  'Software Development Engineer, Amazon Connect',
  'Software Dev Engineer II, Live Events Amazon Advertising',
  'Software Development Engineer II, Amazon Privacy Services',
  'Software Dev Engineer, Amazon',
  'Software Development Engineer/Full Stack developer, Amazon Ads, Campaign Creation, Campaign Creation (Amazon Ads)'],
 'location': 'New York City, NY',
 'currency': 'USD',
 'period': 'YEAR',
 'salary_range': 'None 129300.00 - None 151300.00',
 'base_salary': 'None 129300.00',
 'median_salary': 'None 129300.00',
 'mean_salary': 'None 133700.00',
 'company_rating': None}

# Switching between APIs

In [1]:
import requests

url = "https://jsearch.p.rapidapi.com/search"
headers = {
    "X-RapidAPI-Key": "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

# Print rate limit headers
print("X-RateLimit-Limit:", response.headers.get("X-RateLimit-Limit"))
print("X-RateLimit-Remaining:", response.headers.get("X-RateLimit-Remaining"))
print("X-RateLimit-Reset:", response.headers.get("X-RateLimit-Reset"))


X-RateLimit-Limit: None
X-RateLimit-Remaining: None
X-RateLimit-Reset: None


In [3]:
from pprint import pprint
pprint(response.headers)

{'Date': 'Thu, 29 May 2025 09:08:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '153', 'Connection': 'keep-alive', 'x-amzn-RequestId': 'a841bd62-ca9b-4719-8e26-d2ccabff7b37', 'Access-Control-Allow-Methods': 'GET,OPTIONS', 'Access-Control-Allow-Origin': '*', 'X-Amzn-Trace-Id': 'Root=1-6838242a-705d920a7362b28f408b3fe0;Parent=4b55fa046d0cdb0b;Sampled=0;Lineage=1:b4918fbd:0', 'x-amz-apigw-id': 'LUqW0HoMIAMEC5A=', 'Access-Control-Allow-Headers': 'Content-Type,X-Amz-Date,Authorization,X-Api-Key,X-Amz-Security-Token', 'X-RateLimit-Requests-Limit': '200', 'X-RateLimit-Requests-Remaining': '175', 'X-RateLimit-Requests-Reset': '2405991', 'X-RateLimit-rapid-free-plans-hard-limit-Limit': '500000', 'X-RateLimit-rapid-free-plans-hard-limit-Remaining': '499975', 'X-RateLimit-rapid-free-plans-hard-limit-Reset': '2405991', 'Server': 'RapidAPI-1.2.8', 'X-RapidAPI-Version': '1.2.8', 'X-RapidAPI-Region': 'AWS - ap-southeast-1', 'X-RapidAPI-Request-Id': '1cb6d952eb99e89ca7e0893baf9be6afb46

**The ones useful for us are :-**<BR>
```'X-RateLimit-Requests-Limit': '200'```<br>
```'X-RateLimit-Requests-Remaining': '175'```

## For GlassDoor API

In [5]:
import requests

url = "https://real-time-glassdoor-data.p.rapidapi.com/search"
headers = {
    "X-RapidAPI-Key": "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    "X-RapidAPI-Host": "real-time-glassdoor-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print("Requests Limit:", response.headers.get("x-ratelimit-requests-limit"))
print("Requests Remaining:", response.headers.get("x-ratelimit-requests-remaining"))
print("Requests Reset Time:", response.headers.get("x-ratelimit-requests-reset"))


Requests Limit: None
Requests Remaining: None
Requests Reset Time: None


In [8]:
pprint(response.headers)

{'Date': 'Thu, 29 May 2025 09:21:54 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-RapidAPI-Version': '1.2.8', 'X-RapidAPI-Region': 'AWS - ap-southeast-1', 'X-RapidAPI-Request-Id': '10b9bbcf39747b70ea4f6f1835a3ad87ae0dbf19e77ab514c38f6a09ea0a1d30', 'X-RapidAPI-Proxy-Response': 'true', 'Server': 'RapidAPI-1.2.8'}


In [21]:
def extract_salary(output_dict):
    results = output_dict['data']['aggregateSalaryResponse']['results']
    company_name = next((p['employer']['name'] for p in results),None)
    # employer = result.get('employer', {}).get('name',{}) 
    location = output_dict.get('data',{}).get('aggregateSalaryResponse',{}).get('queryLocation',{}).get('name',{})
    # Initialize results container
    container = []
    # we will traverse through all job entries
    for job in results:
        pay_period = job['payPeriod']
        # we will only process annual salaries
        if pay_period == 'ANNUAL':
            job_title = job['jobTitle']['text']
            currency = job['currency']['code']
            base_mean = job['basePayStatistics']['mean']
            total_additional_mean = job['totalAdditionalPayStatistics']['mean']
            percentiles = {p['ident']: p['value'] for p in job['totalPayStatistics']['percentiles']}
            
            # salary summary
            container.append({
                'company_name': company_name,
                'job_title': job_title,
                'location': location,
                # base salary ranges
                'min_base_salary': round(base_mean * 0.81, 2),  # estimated ~19% below mean
                'median_base_salary': round(base_mean, 2),
                'max_base_salary': round(base_mean * 1.23, 2),  # estimated ~23% above mean
                # general salary ranges
                'min_salary': round(percentiles.get('P25', 0), 2),
                'median_salary': round(percentiles.get('P50', 0), 2),
                'mean_salary': round(total_additional_mean, 2),
                'max_salary': round(percentiles.get('P75', 0), 2),
                'salary_period': 'YEAR',
                'salary_range': f"{currency} {round(percentiles.get('P25', 0), 2)} - {currency} {round(percentiles.get('P75', 0), 2)}",
                'salary_currency': currency
            })
    return container

In [22]:
extract_salary(result)

[{'company_name': 'Apple Inc.',
  'job_title': 'Software Engineer',
  'location': 'New York, NY',
  'min_base_salary': 149867.71,
  'median_base_salary': 185021.86,
  'max_base_salary': 227576.89,
  'min_salary': 228183.85,
  'median_salary': 289326.03,
  'mean_salary': 104304.17,
  'max_salary': 374313.83,
  'salary_period': 'YEAR',
  'salary_range': 'USD 228183.85 - USD 374313.83',
  'salary_currency': 'USD'},
 {'company_name': 'Apple Inc.',
  'job_title': 'Sales Associate',
  'location': 'New York, NY',
  'min_base_salary': 60962.76,
  'median_base_salary': 75262.67,
  'max_base_salary': 92573.08,
  'min_salary': 57027.03,
  'median_salary': 75262.67,
  'mean_salary': 0.0,
  'max_salary': 99329.56,
  'salary_period': 'YEAR',
  'salary_range': 'USD 57027.03 - USD 99329.56',
  'salary_currency': 'USD'},
 {'company_name': 'Apple Inc.',
  'job_title': 'Manager',
  'location': 'New York, NY',
  'min_base_salary': 102433.6,
  'median_base_salary': 126461.23,
  'max_base_salary': 155547.31

In [1]:

def indeed_api(company, job_title, location):
    conn = http.client.HTTPSConnection("indeed-jobs3.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
        'x-rapidapi-host': "indeed-jobs3.p.rapidapi.com"
    }
    query_params = {
        'query': job_title,
        'location': location,
        'company': company
    }
    url_safe_query = urllib.parse.urlencode(query_params)
    conn.request("GET", f"/search?{url_safe_query}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    output_dict = json.loads(data)
    return output_dict


In [5]:
ans = indeed_api('apple', 'software engineer', 'new york')

In [6]:
ans

{'data': {'filters': [{'dataType': 'ATTRIBUTE',
    'filterName': 'remotejob',
    'noOptionValMap': {'remotejob': 'All jobs'},
    'options': [{'@class': 'com.indeed.yosegi.model.dynamicfilters.view.ImmutableBucketOption',
      'count': 515,
      'label': 'Hybrid work',
      'notZrp': True,
      'paramValMap': {'remotejob': '2'},
      'scFilters': [{'@class': 'com.indeed.yosegi.model.dynamicfilters.view.ImmutableSearchCriteriaKeywordFilter',
        'field': 'attr',
        'keys': ['attr_PAXZC'],
        'type': 'keyword'}],
      'selected': False,
      'type': 'bucket',
      'url': None,
      'urlRel': None},
     {'@class': 'com.indeed.yosegi.model.dynamicfilters.view.ImmutableBucketOption',
      'count': 190,
      'label': 'Remote',
      'notZrp': True,
      'paramValMap': {'remotejob': '1'},
      'scFilters': [{'@class': 'com.indeed.yosegi.model.dynamicfilters.view.ImmutableSearchCriteriaKeywordFilter',
        'field': 'attr',
        'keys': ['attr_DSQF7'],
      